In [1]:
!pip install pyLDAvis

    100% |████████████████████████████████| 1.6MB 2.1MB/s ta 0:00:011
    100% |████████████████████████████████| 174kB 2.0MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 1.8MB/s ta 0:00:01
    100% |████████████████████████████████| 829kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 2.0MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 2.3MB/s ta 0:00:011
  Running setup.py bdist_wheel for pyLDAvis ... done
  Stored in directory: /Users/admin/Library/Caches/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/admin/Library/Caches/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built pyLDAvis future


In [9]:
import pandas as pd
import numpy as np

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns

# Model 1: Food and Animals

In [5]:
doc_1 = 'I like to eat broccoli and bananas.'
doc_2 = 'I ate a banana and spinach smoothie for breakfast.'
doc_3 = 'Chinchillas and kittens are cute.'
doc_4 = 'My sister adopted a kitten yesterday.'
doc_5 = 'Look at this cute hamster munching on a piece of broccoli.'

## Step 1: Preprocess our text.

In [19]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # stemming
    porter_stemmer = PorterStemmer()
    
    text_processed = [porter_stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- we're keeping our words distinct

In [20]:
text_process(doc_1)

['like', 'eat', 'broccoli', 'banana']

In [21]:
texts=(text_process(doc_1),
      text_process(doc_2),
      text_process(doc_3),
      text_process(doc_4),
      text_process(doc_5))

In [22]:
texts

(['like', 'eat', 'broccoli', 'banana'],
 ['ate', 'banana', 'spinach', 'smoothi', 'breakfast'],
 ['chinchilla', 'kitten', 'cute'],
 ['sister', 'adopt', 'kitten', 'yesterday'],
 ['look', 'cute', 'hamster', 'munch', 'piec', 'broccoli'])

## Step 2: Fit LDA Model.

In [16]:
from gensim import corpora, models

In [17]:
import pyLDAvis.gensim

In [18]:
pyLDAvis.enable_notebook() # for the visual

In [23]:
np.random.seed(42)

In [24]:
dictionary=corpora.Dictionary(texts)

In [35]:
dictionary


In [26]:
corpus=[dictionary.doc2bow(text) for text in texts] # turn each documents into a bag of words

In [33]:
#instantiate
ldamodel=models.ldamodel.LdaModel(corpus, # pass in our corpus
                                 id2word= dictionary, ## match documents to spot in dict
                                 num_topics=3,# Hyperparameter num of topics
                                 passes=5,
                                 minimum_probability=0.01) #how many times do we run this

In [31]:
for text in texts:
    print(dictionary.doc2bow(text))

[(0, 1), (1, 1), (2, 1), (3, 1)]
[(0, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(8, 1), (9, 1), (10, 1)]
[(10, 1), (11, 1), (12, 1), (13, 1)]
[(1, 1), (9, 1), (14, 1), (15, 1), (16, 1), (17, 1)]


## Step 3: Visualize LDA model.

In [34]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.078851 -0.031907       1        1  53.969894
0     -0.071077 -0.043663       2        1  25.171991
1     -0.007774  0.075570       3        1  20.858114, topic_info=     Category      Freq        Term     Total  loglift  logprob
term                                                           
12    Default  0.000000      sister  0.000000  18.0000  18.0000
11    Default  0.000000       adopt  0.000000  17.0000  17.0000
13    Default  0.000000   yesterday  0.000000  16.0000  16.0000
7     Default  1.000000     spinach  1.000000  15.0000  15.0000
4     Default  1.000000         ate  1.000000  14.0000  14.0000
6     Default  1.000000     smoothi  1.000000  13.0000  13.0000
5     Default  1.000000   breakfast  1.000000  12.0000  12.0000
10    Default  1.000000      kitten  1.000000  11.0000  11.0000
0     Default  1.000000      banana  1.000000  10.0000  10.0000
1     Default  1.000000    broccoli  1.000000   9.0000   9.0000
9     Default  1.000000        cute  1.000000   8.0000   8.0000
8     Default  1.000000  chinchilla  1.000000   7.0000   7.0000
3     Default  1.000000        like  1.000000   6.0000   6.0000
2     Default  1.000000         eat  1.000000   5.0000   5.0000
15    Default  1.000000        look  1.000000   4.0000   4.0000
14    Default  1.000000     hamster  1.000000   3.0000   3.0000
17    Default  1.000000        piec  1.000000   2.0000   2.0000
16    Default  1.000000       munch  1.000000   1.0000   1.0000
1      Topic1  1.460943    broccoli  1.788276   0.4146  -2.0952
9      Topic1  1.459123        cute  1.787831   0.4136  -2.0965
3      Topic1  0.838626        like  1.158870   0.2933  -2.6503
2      Topic1  0.838510         eat  1.158848   0.2932  -2.6504
8      Topic1  0.836582  chinchilla  1.158361   0.2913  -2.6527
15     Topic1  0.829835        look  1.157021   0.2844  -2.6608
14     Topic1  0.829741     hamster  1.157004   0.2843  -2.6609
17     Topic1  0.828507        piec  1.156729   0.2830  -2.6624
16     Topic1  0.827940       munch  1.156619   0.2824  -2.6631
10     Topic1  0.815270      kitten  1.608483  -0.0628  -2.6785
0      Topic1  0.828122      banana  1.654098  -0.0751  -2.6629
11     Topic1  0.211919       adopt  0.984757  -0.9194  -4.0258
...       ...       ...         ...       ...      ...      ...
11     Topic2  0.166787       adopt  0.984757  -0.3962  -3.5026
12     Topic2  0.166720      sister  0.984607  -0.3965  -3.5030
16     Topic2  0.175678       munch  1.156619  -0.5052  -3.4507
17     Topic2  0.175113        piec  1.156729  -0.5085  -3.4539
14     Topic2  0.174302     hamster  1.157004  -0.5134  -3.4586
15     Topic2  0.174195        look  1.157021  -0.5140  -3.4592
8      Topic2  0.167795  chinchilla  1.158361  -0.5526  -3.4966
2      Topic2  0.167213         eat  1.158848  -0.5565  -3.5001
3      Topic2  0.167082        like  1.158870  -0.5573  -3.5009
10     Topic2  0.168582      kitten  1.608483  -0.8762  -3.4919
9      Topic2  0.174531        cute  1.787831  -0.9472  -3.4573
1      Topic2  0.173809    broccoli  1.788276  -0.9516  -3.4614
12     Topic3  0.606488      sister  0.984607   1.0829  -2.0237
11     Topic3  0.606050       adopt  0.984757   1.0820  -2.0244
13     Topic3  0.605995   yesterday  0.984739   1.0819  -2.0245
10     Topic3  0.624632      kitten  1.608483   0.6215  -1.9942
5      Topic3  0.153009   breakfast  1.025960  -0.3355  -3.4009
6      Topic3  0.152957     smoothi  1.025929  -0.3358  -3.4012
4      Topic3  0.152746         ate  1.025938  -0.3372  -3.4026
7      Topic3  0.152664     spinach  1.025928  -0.3377  -3.4031
8      Topic3  0.153984  chinchilla  1.158361  -0.4505  -3.3945
17     Topic3  0.153110        piec  1.156729  -0.4548  -3.4002
16     Topic3  0.153001       munch  1.156619  -0.4554  -3.4009
15     Topic3  0.152992        look  1.157021  -0.4558  -3.4010
14     Topic3  0.152961     hamster  1.

## Step 4: Update model with new data!

In [ ]:
doc_6 = 'That cat is so cute! It looks good enough to eat.'

# Model 2: Yelp Reviews

In [ ]:
review = pd.read_json("./yelp_academic_dataset_review.json")